In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')


In [2]:
datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

training_set = datagen.flow_from_directory(
    "C:/Users/nandu/Downloads/training_set/training_set/",
    target_size=(64, 64),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

datagen1 = ImageDataGenerator(rescale=1./255)

test_set = datagen1.flow_from_directory(
    "C:/Users/nandu/Downloads/test_set/test_set",
    target_size=(64, 64),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(generator):
    features = model.predict(generator)
    features_flattened = features.reshape((features.shape[0], -1))
    return features_flattened

train_features = extract_features(training_set)
test_features = extract_features(test_set)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
251/251 ━━━━━━━━━━━━━━━━━━━━ 76s 302ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 279ms/step


In [4]:
y_train = training_set.classes
y_test = test_set.classes

svm = SVC(kernel='linear')
svm.fit(train_features, y_train)

y_pred = svm.predict(test_features)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Test Accuracy: 0.764211566979733
[[746 265]
 [212 800]]
              precision    recall  f1-score   support

           0       0.78      0.74      0.76      1011
           1       0.75      0.79      0.77      1012

    accuracy                           0.76      2023
   macro avg       0.76      0.76      0.76      2023
weighted avg       0.76      0.76      0.76      2023



In [5]:
import joblib
joblib.dump(svm, 'svm_classifier.pkl')


['svm_classifier.pkl']

In [6]:
from tensorflow.keras.preprocessing import image
import joblib

svm = joblib.load('svm_classifier.pkl')

def predict_image(img_path):
    test_image = image.load_img(img_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image / 255.0
    features = model.predict(test_image)
    features_flattened = features.reshape((features.shape[0], -1))
    result = svm.predict(features_flattened)
    if result[0] == 0:
        print("The image classified is cat")
    else:
        print("The image classified is dog")

predict_image("C:/Users/nandu/Downloads/training_set/training_set/cats/cat.1028.jpg")
predict_image("C:/Users/nandu/Downloads/training_set/training_set/dogs/dog.1077.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
The image classified is cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
The image classified is dog
